# Rescaling (done by switching modulus)

Rescaling(modulus switching)이랑 relinearizaion이랑 다름! 
Modulus switching을 하면 coeff_modulus에서 prime이 하나씩 사라지고, side-effect로 ctxt의 스케일이 prime만큼씩 줄어듬. (scale은 significand의 크기, 즉 계산의 정확도를 결정)  
따라서 coeff_modulus의 prime들을 잘~~ 정해줘야함.  
coeff_modulus = [p1,p2,p3,p4,p5]라고 하면, 한번 곱하기를 할 때마다 /p4 /p3 /p2 /p1 만큼씩 작아짐.  
반대로, ctxt끼리 곱하기가 된다면 둘의 scale만큼 scale이 커짐.  
제곱을 할 경우, scale -> scale^2/p로 바뀜. 이 과정에서 scale이 어느정도 유지되도록 하는게 중요. 

initial scale S와 prime P를 비슷한 크기로 두면 계산 depth가 깊어지면서 scale이 유지될 수 있음. 

### CKKS coeff modulus bits 결정
1. 첫 번째 prime은 60bit 권장. Multiplicative depth를 정확히 맞췄다고 가정할 때, 마지막으로 남는 prime이 이 값임. 이 값이 마지막 ctxt의 scale보다 몇 bit 더 커야 decrypt후의 정확도를 보장할 수 있음. 
2. last prime (special prime)도 60bit 권장. Special prime은 prime중 가장 크거나 같아야함.
3. 중간 prime들은 서로 비슷한 값으로. Scale이 곱해서 커지고, mod switch해서 줄어드는 과정을 거치며 가능한 일정하게 유지. 

예) [60, 40, 40, 40, 60]






### (!) rescale_to_next_inplace Vs.  mod_switch_to_inplace
sss


In [1]:
import numpy as np
from fase import seal

In [2]:
parms = seal.EncryptionParameters(seal.scheme_type.ckks)
poly_modulus_degree = 8192
parms.set_poly_modulus_degree(poly_modulus_degree)
parms.set_coeff_modulus(seal.CoeffModulus.Create(poly_modulus_degree, [60, 40, 40, 60]))

In [3]:
# ciphertext를 40bit 만큼 키울 것.
# Initial coeffModulus가 60bit이므로 40bit은 significand, 20bit은 exponent....?
# 몰라 헷갈려
# encoder.encode에서 scale = float을 기대함. 2**40하면 integer라 에러. 
# 근데 에러가 "E101: Number of dimensions must be one"라고 나와서 헷갈리니 주의. 
scale = 2.**40 
context = seal.SEALContext(parms)

In [4]:
# Generate keys and initiate Agents
keygen = seal.KeyGenerator(context)
secret_key = keygen.secret_key()
public_key = keygen.create_public_key()
relin_keys = keygen.create_relin_keys()
Galoiskeys = keygen.create_galois_keys()

encryptor = seal.Encryptor(context, public_key) # Use public key
evaluator = seal.Evaluator(context)
decryptor = seal.Decryptor(context, secret_key) # Use secret key
encoder   = seal.CKKSEncoder(context)

slot_count = encoder.slot_count()
print("Number of slots:", slot_count)

Number of slots: 4096


Slot number 만큼의 vector 계산을 한번에 할 수 있음. 

In [5]:
vec = np.random.rand(slot_count)
print(vec.shape)

(4096,)


Evaluating polynomial $$\pi \, x^3 + 0.4x + 1$$

In [6]:
p_coeff3 = encoder.encode(np.pi, scale)
p_coeff1 = encoder.encode(0.4, scale)
p_coeff0 = encoder.encode(1., scale)

In [7]:
x_plain = encoder.encode(vec, scale)
x_enc = encryptor.encrypt(x_plain)

In [8]:
# First calculate x^2 (then multiply by x again.)
x3_enc = evaluator.square(x_enc)
evaluator.relinearize_inplace(x3_enc, relin_keys) # relinearization
print("Scale after square", np.log2(x3_enc.scale()))

# Rescale
evaluator.rescale_to_next_inplace(x3_enc)
print("Scale after rescaling", np.log2(x3_enc.scale()))

Scale after square 80.0
Scale after rescaling 40.00000019347918


80bit에서 coeff modulus 하나인 40bit 만큼이 줄어듬. (나누기 2가 아니라 -40임)

In [9]:
x1_coeff3 = evaluator.multiply_plain(x_enc, p_coeff3)
print("Scale of PI * x before rescale", np.log2(x1_coeff3.scale()))
evaluator.rescale_to_next_inplace(x1_coeff3)
print("Scale of PI * x after rescale", np.log2(x1_coeff3.scale()))

Scale of PI * x before rescale 80.0
Scale of PI * x after rescale 40.00000019347918


#### multipying two cipher texts 
두 ciphertext의 스케일과 level이 동일해야 곱하기 가능. 

In [10]:
evaluator.multiply_inplace(x3_enc, x1_coeff3)
evaluator.relinearize_inplace(x3_enc, relin_keys) # c * c 다음엔 relinearize
print("Scale of pi*x3 before rescaling", np.log10(x3_enc.scale()))
evaluator.rescale_to_next_inplace(x3_enc)
print("Scale of pi*x3 after rescaling", np.log10(x3_enc.scale()))

Scale of pi*x3 before rescaling 24.08239976960457
Scale of pi*x3 after rescaling 12.041200234262169


In [11]:
# 1차항 
evaluator.multiply_plain_inplace(x_enc, p_coeff1) 
print("Scale of 0.4x before rescaling", np.log10(x_enc.scale()))
evaluator.rescale_to_next_inplace(x_enc)
print("Scale of 0.4x after rescaling", np.log10(x_enc.scale()))

Scale of 0.4x before rescaling 24.082399653118497
Scale of 0.4x after rescaling 12.041199884802285


#### summation / subtraction 
더하기 빼기 하려면 항들의 **scale**과 encryption parameter가 같아야함.  
**하지만** rescale (modulus switching)을 하느라 서로 다른 mudulus를 사용했음.  (scale도 미묘하게 다름..)

In [13]:
print("Parameters used by all three terms are different")
print("Modulus chain index for x3_enc:", 
     context.get_context_data(x3_enc.parms_id()).chain_index())
print("Modulus chain index for x_enc:", 
     context.get_context_data(x_enc.parms_id()).chain_index())
print("Modulus chain index for p_coeff0:", 
     context.get_context_data(p_coeff0.parms_id()).chain_index())

Parameters used by all three terms are different
Modulus chain index for x3_enc: 0
Modulus chain index for x_enc: 1
Modulus chain index for p_coeff0: 2


#### 지금까지 일어난 일 정리 
- Product x^2 has scale 2^80 and is at level 2;
- Product PI*x has scale 2^80 and is at level 2;
- We rescaled both down to scale 2^80/P_2 and level 1;
- Product PI*x^3 has scale (2^80/P_2)^2;
- We rescaled it down to scale (2^80/P_2)^2/P_1 and level 0;
- Product 0.4*x has scale 2^80;
- We rescaled it down to scale 2^80/P_2 and level 1;
- The contant term 1 has scale 2^40 and is at level 2.

Although the scales of all three terms are approximately 2^40, ***their exact
values are different**, hence they cannot be added together.

In [13]:
print("Exact scales of all three terms are different")
print("Exact scales in PI*x^3 : {:.9f}".format(np.log2(x3_enc.scale())))
print("Exact scales in 0.4x : {:.9f}".format(np.log2(x_enc.scale())))
print("Exact scales in 1 : {:.9f}".format(np.log2(p_coeff0.scale())))

Exact scales of all three terms are different
Exact sccle in PI*x^3 : 40.000001354
Exact sccle in 0.4x : 40.000000193
Exact sccle in 1 : 40.000000000


#### 우선 scale을 맞추려면
1. 미묘하게 다른 scale을 하나로 통일하고, 그만큼의 에러를 감수 
2. 1을 적절한 parameter로 encode하고 0.4x와 마찬가지로 2^80/P_2로 rescale해서 PI*x^3과 scale을 정확히 맞춰줌 (뭐래..?)

여기선 1번 방법 사용 -- neural network도 저정도 에러 무시할 수 있으니 1번으로 하면 될 듯. 

In [14]:
# Scale 강제로 맞추기.
x3_enc.scale(2.**40)
x_enc.scale(2.**40)

In [15]:
# encryption parameter 문제 -> Rescale 없는 modulus switch
print(" Normalize encryption parameters to the lowest level")
last_parms_id = x3_enc.parms_id()
evaluator.mod_switch_to_inplace(x_enc, last_parms_id)
evaluator.mod_switch_to_inplace(p_coeff0, last_parms_id)

 Normalize encryption parameters to the lowest level


In [16]:
# Now all three cipher/plain texts are compatible.
print("Compute PI*x^3 + 0.4*x + 1")
result = evaluator.add(x3_enc, x_enc)
evaluator.add_plain_inplace(result, p_coeff0)

print("Decrypt the result")
plain_result = decryptor.decrypt(result)
decoded_result = encoder.decode(plain_result)
print(decoded_result)

Compute PI*x^3 + 0.4*x + 1
Decrypt the result
[1.37068161 2.7374262  1.35776653 ... 1.13599606 4.19755745 1.48878392]


In [17]:
np.pi*vec**3 + 0.4*vec + 1

array([1.37068139, 2.73742481, 1.35776632, ..., 1.13599601, 4.19755477,
       1.48878361])